In [2]:
import os
import json
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import random

%pip install ultralytics --upgrade --quiet

from ultralytics import YOLO
import torch
from pathlib import Path
import time
import os

#Revisión Imagenes Originales
from pathlib import Path
ORIGINAL_IMAGES = Path('data/ImagenesOriginales/')
if not ORIGINAL_IMAGES.exists():
    raise FileNotFoundError(f"Ruta de datos no encontrada: {ORIGINAL_IMAGES}")

#Revisión Imagenes Filtradas
from pathlib import Path
FILTERED_IMAGES = Path('data/ImagenesFiltradas/')
if not FILTERED_IMAGES.exists():
    raise FileNotFoundError(f"Ruta de datos no encontrada: {FILTERED_IMAGES}")

DATA_ROOT = Path('data/')
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Ruta de datos no encontrada: {DATA_ROOT}")

root = DATA_ROOT
train_img_orig = root / 'ImagenesOriginales' / 'train'
train_img_fil = root / 'ImagenesFiltradas' / 'train'
train_lbl = root / 'EtiquetasProcesadas' / 'train'

# Definir clases
class_names = [
    'carretera', 'rio'
]

##Tabla de conteo por grupo
counts = {}
for split in ['ImagenesFiltradas/train/images', 'ImagenesFiltradas/val/images', 'ImagenesFiltradas/test/images']:
    path = root / split 
    counts[split] = len(list(path.glob('*.*'))) if path.exists() else 0

# Mostrar tabla de distribución
dist = pd.Series(counts, name='count')
print(dist.to_frame())


You should consider upgrading via the 'c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.
                                count
ImagenesFiltradas/train/images    610
ImagenesFiltradas/val/images       74
ImagenesFiltradas/test/images     226


In [3]:
# Load a pre-trained YOLOv11 model
model = YOLO('yolo11l.pt')  

print("Model loaded successfully")

Model loaded successfully


In [4]:
# Freeze the backbone - only train the head
model.model.freeze = ['model.0', 'model.1', 'model.2', 'model.3', 'model.4', 'model.5', 'model.6', 
                       'model.7', 'model.8', 'model.9', 'model.10', 'model.11', 'model.12', 'model.13']

# Count trainable vs total parameters
trainable_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.model.parameters())
print(f"Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params/total_params:.1%})")

Trainable parameters: 0 / 25,372,160 (0.0%)


In [ ]:
#Modelo Originales

# Define paths
root = Path('data')
yaml_path = root / 'rio-carretera-differentiation_orig.yaml'

#Fine Tune / Correr Modelo
print("Starting rapid fine-tuning...")
start_time = time.time()

# Create output directory
os.makedirs('runs/detect', exist_ok=True)

# Train with minimal epochs and a small batch size for quick results
results = model.train(
    data=str(yaml_path),              # Path to your data YAML
    epochs=20,                         # Very few epochs for fast adaptation
    imgsz=640,                        # Image size
    batch=8,                          # Smaller batch size
    patience=3,                       # Early stopping patience
    name='rapid_finetune',            # Run name
    pretrained=True,                  # Use pretrained weights
    freeze=14,                        # Alternative way to freeze layers
    optimizer='Adam',                 # Adam optimizer converges faster for fine-tuning
    lr0=0.001,                        # Lower learning rate for fine-tuning
    device='0' if torch.cuda.is_available() else 'cpu' # Use GPU if available
)

end_time = time.time()
print(f"Fine-tuning completed in {end_time - start_time:.2f} seconds")

# Count trainable vs total parameters
trainable_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.model.parameters())
print(f"Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params/total_params:.1%})")

# Save the model
model.export(format="onnx")  # Export to ONNX format for deployment
model.save('model3_Orig.pt')
print(f"model3_Orig.pt")

Starting rapid fine-tuning...
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CUDA:0 (Quadro T1000, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\rio-carretera-differentiation_orig.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=14, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model2.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rapid_finetune10, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, 

train: Scanning C:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\notebooks\data\ImagenesOriginales\train\labels.cache... 610 images, 0 backgrounds, 0 corrupt: 100%|██████████| 610/610 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 485.767.9 MB/s, size: 66.8 KB)


val: Scanning C:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\notebooks\data\ImagenesOriginales\val\labels.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]


Plotting labels to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

       7/20      3.99G      2.104      2.317       2.12          8        640: 100%|██████████| 77/77 [02:28<00:00,  1.93s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:10<00:00,  2.03s/it]

                   all         74        124      0.619      0.524      0.516      0.179
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 4, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



7 epochs completed in 0.497 hours.
Optimizer stripped from c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10\weights\last.pt, 51.2MB
Optimizer stripped from c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10\weights\best.pt, 51.2MB

Validating c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10\weights\best.pt...
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CUDA:0 (Quadro T1000, 4096MiB)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:08<00:00,  1.75s/it]


                   all         74        124       0.52      0.542      0.484      0.195
             carretera         33         62      0.441      0.435      0.336      0.142
                   rio         45         62      0.598      0.649      0.632      0.247
Speed: 1.1ms preprocess, 111.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10
Fine-tuning completed in 1830.21 seconds
Trainable parameters: 0 / 25,312,022 (0.0%)
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CPU (Intel Core(TM) i7-10850H 2.70GHz)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs

PyTorch: starting from 'c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune10\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (48.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 

In [17]:
modelo = 'model3_Orig.pt'

# IMPORTANT: Load your fine-tuned model instead of the pre-trained one
model = YOLO(modelo)  # Cambiar acá según el modelo a aplicar

print("Modelo cargado:", modelo)
print("\nMapeo de clases del modelo:")
for class_id, class_name in model.names.items():
    print(f"  Class {class_id}: {class_name}")

# Evaluar el modelo en el conjunto de validación
print("\nEvaluando el modelo fine-tuned...")
results = model.val(data=str(root / 'rio-carretera-differentiation_orig.yaml'))
# Manejar arrays numpy para precision y recall
if hasattr(results.box.p, 'mean'):
    precision_mean = float(results.box.p.mean())
    recall_mean = float(results.box.r.mean())
else:
    precision_mean = float(results.box.p)
    recall_mean = float(results.box.r)
print(f"  Precision: {precision_mean:.4f} ({precision_mean*100:.2f}%)")
print(f"  Recall:    {recall_mean:.4f} ({recall_mean*100:.2f}%)")
# Calcular F1-Score
if precision_mean > 0 and recall_mean > 0:
    f1_score = 2 * (precision_mean * recall_mean) / (precision_mean + recall_mean)
    print(f"  F1-Score:  {f1_score:.4f} ({f1_score*100:.2f}%)")
else:
    print(f"  F1-Score:  No calculable")
    

Modelo cargado: model3_Orig.pt

Mapeo de clases del modelo:
  Class 0: carretera
  Class 1: rio

Evaluando el modelo fine-tuned...
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CUDA:0 (Quadro T1000, 4096MiB)
YOLO11l summary (fused): 190 layers, 25,280,854 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 480.075.9 MB/s, size: 57.9 KB)


val: Scanning C:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\notebooks\data\ImagenesOriginales\val\labels.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.20s/it]


                   all         74        124       0.52      0.542      0.484      0.195
             carretera         33         62      0.441      0.435      0.336      0.142
                   rio         45         62      0.598      0.649      0.632      0.247
Speed: 1.4ms preprocess, 31.3ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\val38
  Precision: 0.5198 (51.98%)
  Recall:    0.5422 (54.22%)
  F1-Score:  0.5307 (53.07%)


In [18]:
#Modelo Filtradas

# Define paths
root = Path('data')
yaml_path = root / 'rio-carretera-differentiation.yaml'

#Fine Tune / Correr Modelo
print("Starting rapid fine-tuning...")
start_time = time.time()

# Create output directory
os.makedirs('runs/detect', exist_ok=True)

# Train with minimal epochs and a small batch size for quick results
results = model.train(
    data=str(yaml_path),              # Path to your data YAML
    epochs=20,                         # Very few epochs for fast adaptation
    imgsz=640,                        # Image size
    batch=8,                          # Smaller batch size
    patience=3,                       # Early stopping patience
    name='rapid_finetune',            # Run name
    pretrained=True,                  # Use pretrained weights
    freeze=14,                        # Alternative way to freeze layers
    optimizer='Adam',                 # Adam optimizer converges faster for fine-tuning
    lr0=0.001,                        # Lower learning rate for fine-tuning
    device='0' if torch.cuda.is_available() else 'cpu' # Use GPU if available
)

end_time = time.time()
print(f"Fine-tuning completed in {end_time - start_time:.2f} seconds")

# Count trainable vs total parameters
trainable_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.model.parameters())
print(f"Trainable parameters: {trainable_params:,} / {total_params:,} ({trainable_params/total_params:.1%})")

# Save the model
model.export(format="onnx")  # Export to ONNX format for deployment
model.save('model3.pt')
print(f"model3.pt")

Starting rapid fine-tuning...
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CUDA:0 (Quadro T1000, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\rio-carretera-differentiation.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=14, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model3_Orig.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rapid_finetune11, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, 

train: Scanning C:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\notebooks\data\ImagenesFiltradas\train\labels.cache... 610 images, 0 backgrounds, 0 corrupt: 100%|██████████| 610/610 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 1380.9259.6 MB/s, size: 378.4 KB)


val: Scanning C:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\notebooks\data\ImagenesFiltradas\val\labels.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]


Plotting labels to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

       5/20      4.01G      1.978      2.155      2.026          3        640: 100%|██████████| 77/77 [01:47<00:00,  1.40s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:14<00:00,  2.96s/it]

                   all         74        124      0.608      0.532      0.524       0.24
EarlyStopping: Training stopped early as no improvement observed in last 3 epochs. Best results observed at epoch 2, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



5 epochs completed in 0.210 hours.
Optimizer stripped from c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11\weights\last.pt, 51.2MB
Optimizer stripped from c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11\weights\best.pt, 51.2MB

Validating c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11\weights\best.pt...
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CUDA:0 (Quadro T1000, 4096MiB)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.46it/s]


                   all         74        124      0.719      0.476      0.517      0.249
             carretera         33         62      0.667      0.419      0.448      0.239
                   rio         45         62      0.771      0.532      0.586      0.259
Speed: 0.5ms preprocess, 37.2ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11
Fine-tuning completed in 792.87 seconds
Trainable parameters: 0 / 25,312,793 (0.0%)
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CPU (Intel Core(TM) i7-10850H 2.70GHz)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs

PyTorch: starting from 'c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\rapid_finetune11\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (48.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'o

In [5]:
modelo = 'model3.pt'

# IMPORTANT: Load your fine-tuned model instead of the pre-trained one
model = YOLO(modelo)  # Cambiar acá según el modelo a aplicar

print("Modelo cargado:", modelo)
print("\nMapeo de clases del modelo:")
for class_id, class_name in model.names.items():
    print(f"  Class {class_id}: {class_name}")

# Evaluar el modelo en el conjunto de validación
print("\nEvaluando el modelo fine-tuned...")
results = model.val(data=str(root / 'rio-carretera-differentiation.yaml'))
# Manejar arrays numpy para precision y recall
if hasattr(results.box.p, 'mean'):
    precision_mean = float(results.box.p.mean())
    recall_mean = float(results.box.r.mean())
else:
    precision_mean = float(results.box.p)
    recall_mean = float(results.box.r)
print(f"  Precision: {precision_mean:.4f} ({precision_mean*100:.2f}%)")
print(f"  Recall:    {recall_mean:.4f} ({recall_mean*100:.2f}%)")
# Calcular F1-Score
if precision_mean > 0 and recall_mean > 0:
    f1_score = 2 * (precision_mean * recall_mean) / (precision_mean + recall_mean)
    print(f"  F1-Score:  {f1_score:.4f} ({f1_score*100:.2f}%)")
else:
    print(f"  F1-Score:  No calculable")
    

Modelo cargado: model3.pt

Mapeo de clases del modelo:
  Class 0: carretera
  Class 1: rio
  Class 2: background

Evaluando el modelo fine-tuned...
Ultralytics 8.3.141  Python-3.9.12 torch-2.2.0+cu121 CUDA:0 (Quadro T1000, 4096MiB)
YOLO11l summary (fused): 190 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2239.2566.3 MB/s, size: 389.0 KB)


val: Scanning C:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\notebooks\data\ImagenesFiltradas\val\labels.cache... 74 images, 0 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.27s/it]


                   all         74        124      0.719      0.476      0.517      0.249
             carretera         33         62      0.667      0.419      0.448      0.239
                   rio         45         62      0.771      0.532      0.586      0.259
Speed: 1.2ms preprocess, 32.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to c:\Users\Camilo Daza\Documents\GitHub\isis4825-proyecto-final\runs\detect\val41
  Precision: 0.7190 (71.90%)
  Recall:    0.4757 (47.57%)
  F1-Score:  0.5726 (57.26%)
